In [15]:
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

np.random.seed(42)

In [16]:
# Create an ar(2) process

#X_t = 0.33 * X_t-1 + 0.5 * X_t-2

#X_t - 0.33 * X_t-1 - 0.5 * X_t-2 = 0

ar2_ex_ar_coefficients = np.array(
    [1, -0.33, -0.5]
)

ar2_ex_ma_coefficients = np.array(
    [1, 0, 0]
)

AR2_process(ArmaProcess(ar2_ex_ar_coefficients,
                        ar2_ex_ma_coefficients).generate_sample(nsample = 1000))

NameError: name 'AR2_process' is not defined

In [ ]:
#Helper function to plot TS along with its PACF and ACF
def plot_ts_pacf_acf(ts, title="TITLE GOES HERE", xlabel = "Time"): 
    global fig, axs
    fig, axs = plt.subplots(3, 1, figsize=(15,12))
    fig.tight_layout(pad = 3.0)
    axs[0].set_xlabel(xlabel)
    axs[0].set_title(title)
    axs[0].plot(ts)
    plot_acf(ts, lags=20, ax=axs[1]);
    plot_pacf(ts, lags=20, ax=axs[2])

In [ ]:
plot_ts_pacf_acf(AR2_process,"PIDOOMA'ed AR(2) Process")

In [ ]:
#Create an MA(2) process

ma2_ex_ar_coefficients = np.array([1]) #AR part...no AR

ma2_ex_ma_coefficients = np.array([1, -0.4, -0.7]) #mA(2) part

MA2_process = (ArmaProcess(ma2_ex_ar_coefficients, ma2_ex_ma_coefficients).generate_sample(nsample=1000)
               

In [ ]:
plot_ts_pacf_acf(MA2_process,"MA(2) Process via PIDOOMA")

In [ ]:
#ARMA(2,2)

ARMA_2_2_process = (ArmaProcess(ar2_ex_ar_coefficients,ma2_ex_ma_coefficients).generate_sample(nsample=1000))

In [ ]:
plot_ts_pacf_acf(ARMA_2_2_process,"ARMA (2,2) Process")

In [5]:
#Stationarity

#particular test for time series
from statsmodels.tsa.stattools import adfuller

result = adfuller(ARMA_2_2_process)

print(result)

#null_hypothesis = not stationary, definition?
print(f'ADF statistic:{result[0]}') #ANS: -17.776 --> negative --> can reject null 
print(f'p-value: {result[1]}') #ANS: 3.291e-30 --> that's good

NameError: name 'ARMA_2_2_process' is not defined

In [7]:
 #An ARIMA(p, d, q) example

import pandas as pd

In [11]:
df = pd.read_csv('./data/jj.csv')
df.head()

,date,data
0,1960-01-01,0.71
1,1960-04-01,0.63
2,1960-07-02,0.85
3,1960-10-01,0.44
4,1961-01-01,0.61


In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

ax.plot(df.data,df['data'])
ax.set_xlabel('Date')
ax.set_ylabel('Earnings per share (USD)')

#quarterly earnings
#every 2 years
plt.xticks(np.arange(0, 81, 8), [1960, 1962, 1964, 1966, 1968, 1970, 1972, 1974, 1976, 1978, 1980] )

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
def adf_analyze(data):
    adf_results = adfuller(data)
    print(f'ADF statistic:{adf_result[0]}') #ANS: -17.776 --> negative --> can reject null 
    print(f'p-value: {adf_result[1]}') #ANS: 3.291e-30 --> that's good

In [ ]:
adf_analyze(df['data']) #2.742

In [ ]:
earnings_diff_1 = np.diff(df['data'],n = 1)

adf_analyze(earnings_diff_1) #-0.407

In [ ]:
earnings_diff_2 = np.diff(earnings_diff_1 ,n = 1)

adf_analyze(earnings_diff_2) #-3.585

In [27]:
d = 2 #d from ARIMA(p, d, q)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

ax.plot(df.data,df['data'])
ax.set_xlabel('Date')
ax.set_ylabel('Earnings per share (USD)')

ax.axvspan(80,83, color="gray", alpha=0.3)

#quarterly earnings
#every 2 years
plt.xticks(np.arange(0, 81, 8), [1960, 1962, 1964, 1966, 1968, 1970, 1972, 1974, 1976, 1978, 1980] )

fig.autofmt_xdate()
plt.tight_layout()

In [31]:
from tqdm.notebook import tqdm_notebook

from statsmodels.tsa.statespace.sarimax import SARIMAX

#Seasonal Autoregressive Integrated Moving Average with eXogenous regressors

from typing import Union

def optimize_ARIMA(endog: Union[pd.Series,list], #encodes the type so either in the union
                   order_list: list,
                   d: int) -> pd.DataFrame:  #value of d for differencing
    results = []
    for order in tqdm_notebook(order_list):
        try: 
            p = order[0]
            q = order[1]
            model = SARIMAX(endog, 
                            order = (p, d, q), 
                            simple_differencing = False).fit(disp = False)
        except: 
            continue

        aic = model.aic
        results.append( [order,aic])

    result_df = pd.DataFrame(results)
    result_df.columns['(p,q)', 'AIC']

    result_df = result_df.sort_Values(by = 'AIC', ascending = True).reset_index(drop = True)

    return result_df

In [15]:
ps = range(0, 4, 1)
qs = range(0, 4, 1)

#we found d above by repeating the ADF test and differencing and differentiating 

In [21]:
from itertools import product
train = df['data'][:-4] #set aside last 4 quarters, train on rest
order_list = product(ps, qs)

In [23]:
list(order_list)

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3)]

In [33]:
result_df = optimize_ARIMA(train, order_list, d)

0it [00:00, ?it/s]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [35]:
result_df

#(p, q) = (3, 3) AIC = 115.266 #lower AIC is better

NameError: name 'result_df' is not defined

In [ ]:
model = SARIMAX(train, order = (3, 2, 3), simple_differencing = False)

model_fit = model.fit(disp = False)

print(model_fit.summary())

#Ljung-Box: swedish

In [ ]:
model_fit.plot_diagnostics(figsize=(15,10));

#residuals around 0 without trends or patterns, shouldn't grow from distance
#histogram, green is normal
#QQ
#correlogram: no correlation in residuals, most in blue box --> good


In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

residuals = model_fit.resid

ljb_results = acorr_ljungbox(residuals,np.arange(1,11,1))

ljb_results

In [ ]:
test = df.iloc[-4:]

test['naive_seasonal'] = df['data'].iloc[76:80].values

test

In [ ]:
ARIMA_pred = model_fit.get_prediction(80,83).predicted_mean

test['ARIMA_pred'] = ARIMA_pred

test

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

ax.plot(df.data,df['data'])
ax.plot(test['data'], 'b-', label='actual')
ax.plot(test['naive_seasonal'], 'r:', label='naive seasonal')
ax.plot(test['ARIMA_pred'], 'k--', label='ARIMA(3,2,3)')

ax.set_xlabel('Date')
ax.set_ylabel('Earnings per share (USD)')

ax.axvspan(80,83, color="gray", alpha=0.3)

ax.legend(loc=2)

#quarterly earnings
#every 2 years
plt.xticks(np.arange(0, 81, 8), [1960, 1962, 1964, 1966, 1968, 1970, 1972, 1974, 1976, 1978, 1980] )

fig.autofmt_xdate()
plt.tight_layout()